# 2. 변수 전처리 & EDA

+ 결측치 제거
+ 분석 단위인 격자 별 데이터로 변환

## 0. 교통사고 발생 건수

In [21]:
# 격자별 교통사고 발생 건수
data2 = gpd.read_file('2.오산시_어린이교통사고_격자.geojson')

In [22]:
# 도로와 겹치지 않는 격자 제거
data23 = gpd.read_file('23.오산시_상세도로망_LV6.geojson')

In [23]:
road = data23.geometry.unary_union
#filter1 = [geo.intersects(road) for geo in data2.geometry[:10]]

filter1 = []
for index, row in data2.iterrows():
    filter1.append(row.geometry.intersects(road))

len(filter1)

4510

In [24]:
#도로와 겹치는 구간만을 표기하는 filter1
#도로가 없는데 어린이 교통사고가 일어나는 곳
data2[~np.array(filter1)][data2[~np.array(filter1)].accident_cnt > 0].describe()
#다발구간은 없으므로 제거하기로 한다

,accident_cnt
count,49.000000
mean,1.122449
std,0.389051
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,3.000000


In [25]:
data2 = data2[filter1]

In [26]:
# 교통사고 관련 요인
data1 = pd.read_csv('1.오산시_주정차단속(2018~2020).csv') #1. parking
data3 = gpd.read_file('3.오산시_차량등록현황_격자.geojson') #2. car_cnt
data4 = gpd.read_file('4.오산시_연령별_거주인구격자(총인구).geojson') #3. pop_whole
data5 = gpd.read_file('5.오산시_연령별_거주인구격자(유소년).geojson') #4 pop_kid
data17 = gpd.read_file('17.오산시_횡단보도.geojson') #5 crswlk
data19 = gpd.read_file('19.오산시_신호등.geojson') #6 trf_lit
data23 = gpd.read_file('23.오산시_상세도로망_LV6.geojson')
data24 = pd.read_csv('24.평일_전일,시간대별_오산시_추정교통량_Level6.csv') # 7.trf_jam


data15 = pd.read_csv('15.오산시_무인교통단속카메라.csv') #8 trf_cam
data16 = pd.read_csv('16.오산시_도로안전표지표준데이터.csv') #9 trf_sign
data18 = pd.read_csv('18.오산시_과속방지턱표준데이터.csv') #10 spd_bmp
data20 = pd.read_csv('20.오산시_CCTV설치현황.csv') #11 cctv

data8 = pd.read_csv("8.오산시_유동인구(2019).csv")
data27 = gpd.read_file('27.오산시_도로명주소_건물.geojson')

/opt/app-root/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


##  1. 불법 주정차

In [27]:
data1.isna().any()
#위도 경도에 결측치 존재
data1[data1.단속위치_경도.isna()].단속위치.unique()
#유탑유블레스 남측에서 단속 [37.1413500, 127.072400]
data1.fillna({"단속위치_경도":127.072400, "단속위치_위도":37.1413500}, inplace=True)

In [28]:
data1_geo = gpd.GeoDataFrame(data1, geometry = gpd.points_from_xy(data1.단속위치_경도, data1.단속위치_위도))
data1_geo["parking"] = 1
data1_geo.crs = data2.crs

In [29]:
data1_geo

,단속일자,단속위치,행정구역,스쿨존여부,단속위치_경도,단속위치_위도,단속종류,geometry,parking
0,20181101,금암동 여계산로 부근,경기도 오산시 세교동,N,127.043462,37.170237,이동형CCTV단속,POINT (127.04346 37.17024),1
1,20181101,세교동 세마역로19번길,경기도 오산시 세교동,N,127.044770,37.181695,이동형CCTV단속,POINT (127.04477 37.18169),1
2,20181101,세교동 수목원로 부근,경기도 오산시 양산동,N,127.042200,37.186357,이동형CCTV단속,POINT (127.04220 37.18636),1
3,20181101,오산동 오산로272번길 부,경기도 오산시 원동,N,127.071732,37.150557,이동형CCTV단속,POINT (127.07173 37.15056),1
4,20181101,오산동 성호대로93번길,경기도 오산시 부산동,N,127.071757,37.150457,이동형CCTV단속,POINT (127.07176 37.15046),1
...,...,...,...,...,...,...,...,...,...
61696,20201111,GS25궐동대호점 맞은편,경기도 오산시 청학동,N,127.053064,37.158140,고정형CCTV단속,POINT (127.05306 37.15814),1
61697,20201111,물향기지하도,경기도 오산시 원동,N,127.066157,37.171143,고정형CCTV단속,POINT (127.06616 37.17114),1
61698,20201111,운암2단지 근린공원삼거,경기도 오산시 청호동,N,127.077283,37.158126,고정형CCTV단속,POINT (127.07728 37.15813),1
61699,20201111,궐동 양평해장국 사거리,경기도 오산시 궐동,N,127.056665,37.155827,고정형CCTV단속,POINT (127.05666 37.15583),1


In [30]:
merge_a = gpd.sjoin(data1_geo, data2, op = "within", how = "right")
parking = merge_a.groupby(['gid']).agg({'parking': 'sum'})

## 2. 차량 등록대수

In [31]:
# 2. 차량등록현황

car_cnt = pd.merge(data3, data2, on="gid", how= "right")["car_cnt"]
#격자별 데이터로 변환

## 3-4. 거주 인주

In [32]:
# 3. 총 거주인구 & 4. 유소년 거주 인구
data4.columns = ["gid", "pop_whole", "geometry"]
data5.columns = ["gid", "pop_kid", "geometry"]
#격자 형태의 데이터, 단 격자 주위의 데이터도 고려할 필요

In [33]:
# 총인구
data4_cen = data4.copy()
data4_cen["geometry"] = data4_cen["geometry"].centroid
data4_cen["count"] = 1
data4_cen.crs = data2.crs

data4_buf = data4.copy()
data4_buf["geometry"] = data4_cen["geometry"].buffer(0.0015) # 주위 격자를 포함할수 있도록 buffer
data4_buf.head()
data4_buf.crs = data2.crs

merge4 = gpd.sjoin(data4_cen, data4_buf.drop("pop_whole",axis =1), op = "within", how = "right")
data4_new =  merge4.groupby(['gid_y']).agg({"pop_whole": "sum", 'count': 'sum', "geometry": "first"})

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [34]:
data4_cen

,gid,pop_whole,geometry,count
0,다사551085,NaN,POINT (126.99478 37.17464),1
1,다사551086,NaN,POINT (126.99477 37.17554),1
2,다사551087,NaN,POINT (126.99476 37.17644),1
3,다사551088,NaN,POINT (126.99476 37.17734),1
4,다사552085,NaN,POINT (126.99590 37.17464),1
...,...,...,...,...
4505,다사646052,NaN,POINT (127.10194 37.14530),1
4506,다사646053,NaN,POINT (127.10194 37.14620),1
4507,다사647050,NaN,POINT (127.10308 37.14350),1
4508,다사647051,NaN,POINT (127.10308 37.14440),1


In [35]:
#유소년 인구
data5_cen = data5.copy()
data5_cen["geometry"] = data5_cen["geometry"].centroid
data5_cen["count"] = 1
data5_cen.crs = data2.crs

data5_buf = data5.copy()
data5_buf["geometry"] = data5_cen["geometry"].buffer(0.0015)
data5_buf.head()
data5_buf.crs = data2.crs

merge5 = gpd.sjoin(data5_cen, data5_buf.drop("pop_kid",axis =1), op = "within", how = "right")

data5_new =  merge5.groupby(['gid_y']).agg({"pop_kid": "sum", 'count': 'sum', "geometry": "first"})

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [36]:
pop_whole = pd.merge(data4_new,data2, left_on="gid_y", right_on = "gid", how = "right")["pop_whole"]
pop_kid = pd.merge(data5_new,data2, left_on="gid_y", right_on = "gid", how = "right")["pop_kid"]

## 5. 횡단보도

In [37]:
data17 = gpd.read_file('17.오산시_횡단보도.geojson') #5 crswlk

In [38]:
# 5. 횡단보도

#다각형을 중심점 으로 바꿔줌
data17_p  = data17.copy()
data17_p["crswlk"] = 1
data17_p['geometry']  = data17_p['geometry'].centroid
data17_p.crs = data2.crs

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [39]:
data17_p

,OBJECTID,geometry,crswlk
0,2,POINT (127.06174 37.15702),1
1,3,POINT (127.03179 37.19500),1
2,4,POINT (127.05529 37.15642),1
3,5,POINT (127.04937 37.13433),1
4,6,POINT (127.06222 37.14971),1
...,...,...,...
1200,1292,POINT (127.07844 37.13977),1
1201,1294,POINT (127.08992 37.15084),1
1202,1295,POINT (127.05784 37.15206),1
1203,1297,POINT (127.04294 37.18184),1


In [40]:
#격자 단위로 전환
merge_b = gpd.sjoin(data17_p, data2, op = "within", how = "right")
crswlk = merge_b.groupby('gid').agg({'crswlk': 'sum'})

## 6. 신호등

In [41]:
# 6. 신호등

#격자 단위로 전환
data19["trf_lit"] = 1
merge_c = gpd.sjoin(data19, data2, op = "within", how = "right")
trf_lit = merge_c.groupby('gid').agg({'trf_lit': 'sum'})

## 7. 도로 교통량

+ 시간 단위로 측정됨
+ 가장 유의한 시간대로 판단되는 오후 1~4시 위주의 격자별 교통량 변수 생성

In [42]:
data24["link_id"] = data24["상세도로망_LinkID"].apply(lambda x: str(x)[:-2])

In [43]:
t8_9 = [a == '9' for a in data24.시간적범위]
t13_16 = [b in ['13','14','15'] for b in data24.시간적범위] # 1~4시
t16_19 = [c in ['16','17','18'] for c in data24.시간적범위] # 4~7시

data24_8to9 = data24[t8_9]
data24_13to16 = data24[t13_16]
data24_16to19 = data24[t16_19]
data24_all = data24[data24.시간적범위 == "전일"]

In [44]:
data23.drop(data23[data23.road_rank == "101"].index, inplace = True) #고속도로 제거

In [45]:
merge_13to16 = pd.merge(data23, data24_13to16, on = "link_id", how ="left") # 상세 도로 데이터와 병합
data23_13to16 = merge_13to16.groupby('link_id').agg({"max_speed":"first", "up_lanes":"first", "dw_lanes":"first", 
                                                     "oneway":"first","전체 추정교통량":"mean",
                                                     "승용차 추정교통량":"mean", "버스 추정교통량":"mean","화물차 추정교통량":"mean",
                                                    "geometry":"first"})

data23_13to16.dropna(inplace = True)
data23_13to16.iloc[:,0:4] = data23_13to16.iloc[:,0:4].apply(lambda x: pd.to_numeric(x))
data23_13to16 = gpd.GeoDataFrame(data23_13to16, geometry = data23_13to16.geometry)
data23_13to16.crs = data2.crs

join13to16 = gpd.sjoin(data23_13to16, data2, op= "intersects", how = "right")
trf_jam = join13to16.groupby("gid").agg("max") #격자내 도로 중 최대 교통량

## 8. 무인 교통단속카메라

In [46]:
data15_geo = gpd.GeoDataFrame(data15, geometry= gpd.points_from_xy(data15.설치위치_경도, data15.설치위치_위도))
data15_geo["trf_cam"] = 1
data15_geo.crs = data2.crs

In [47]:
merge_d = gpd.sjoin(data15_geo, data2, op = "within", how = "right")
trf_cam = merge_d.groupby('gid').agg({'trf_cam': 'sum'})

## 9. 도로안전표지표준데이터

+ 도로안전표지는 표지종 분류 필요성
+ 억제요인으로서의 표지(ex 감속) / 그 외(시선 분산요인) 분류

In [48]:
trf_sign_p_list = [s in ['100m 출구감속표지','200m 출구감속표지','300m 출구감속표지','정류장표지'] for s in data16.표지종별]
trf_sign_n_list = [s not in ['100m 출구감속표지','200m 출구감속표지','300m 출구감속표지','정류장표지'] for s in data16.표지종별]

In [49]:
data16_geo = gpd.GeoDataFrame(data16, geometry= gpd.points_from_xy(data16.lon, data16.lat))
data16_geo.crs = data2.crs
data16_geo_p = data16_geo[trf_sign_p_list] #긍정 요인
data16_geo_p["trf_sign_p"] = 1
data16_geo_n = data16_geo[trf_sign_n_list] #부정 요인
data16_geo_n["trf_sign_n"] = 1

/opt/app-root/lib/python3.6/site-packages/geopandas/geodataframe.py:1190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [50]:
merge_e = gpd.sjoin(data16_geo_p, data2, op = "within", how = "right")
trf_sign_p = merge_e.groupby('gid').agg({'trf_sign_p': 'sum'})

merge_f = gpd.sjoin(data16_geo_n, data2, op = "within", how = "right")
trf_sign_n = merge_f.groupby('gid').agg({'trf_sign_n': 'sum'})

## 10. 과속 방지턱

In [51]:
data18_geo = gpd.GeoDataFrame(data18, geometry= gpd.points_from_xy(data18.설치위치_경도, data18.설치위치_위도))
data18_geo["spd_bmp"] = 1
data18_geo.crs = data2.crs

In [52]:
merge_g = gpd.sjoin(data18_geo, data2, op = "within", how = "right")
spd_bmp = merge_g.groupby('gid').agg({'spd_bmp': 'sum'})

## 11. CCTV

+ cctv역시 유형에 따라 분류하여 변수 생성

In [53]:
data20_geo = gpd.GeoDataFrame(data20, geometry= gpd.points_from_xy(data20.설치위치_경도, data20.설치위치_위도))
data20_geo["cctv"] = 1
data20_geo.crs = data2.crs

In [54]:
# 유형 A : 방범
merge_ha = gpd.sjoin(data20_geo[data20_geo["CCTV 유형코드"] == 'A'], data2, op = "within", how = 'right')
cctv_A = merge_ha.groupby('gid').agg({'cctv': 'sum'})
cctv_A.rename(columns={'cctv': 'cctv_A'})

# 유형 B : 도시공원
merge_hb = gpd.sjoin(data20_geo[data20_geo["CCTV 유형코드"] == 'B'], data2, op = "within", how = 'right')
cctv_B = merge_hb.groupby('gid').agg({'cctv': 'sum'})
cctv_B.rename(columns={'cctv': 'cctv_B'},inplace=True)

# 유형 C : 어린이보호
merge_hc = gpd.sjoin(data20_geo[data20_geo["CCTV 유형코드"] == 'C'], data2, op = "within", how = 'right')
cctv_C = merge_hc.groupby('gid').agg({'cctv': 'sum'})
cctv_C.rename(columns={'cctv': 'cctv_C'},inplace=True)

# 유형 D : 차량방범
merge_hd = gpd.sjoin(data20_geo[data20_geo["CCTV 유형코드"] == 'D'], data2, op = "within", how = 'right')
cctv_D = merge_hd.groupby('gid').agg({'cctv': 'sum'})
cctv_D.rename(columns={'cctv': 'cctv_D'},inplace=True)